In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [3]:
X, y = make_classification(
    n_samples=1000,
    n_features=10,
    n_informative=2,
    n_redundant=8,
    flip_y = 0.1,
    random_state=42,
    weights= [0.9, 0.1]
)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.3, stratify=y ,random_state=42)

In [5]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    )
]

In [6]:
reports = []

for model_name, params,  model, train_set , test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]

    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [7]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [8]:
mlflow.set_experiment("Anomaly Detection")
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]

    with mlflow.start_run(run_name=model_name):
        for key, value in params.items():
            mlflow.log_param(key, value)
        
        mlflow.log_metrics({
            'accuracy':report['accuracy'],
            'recall_class_1':report['1']['recall'],
            'recall_class_0':report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, 'model')
        else:
            mlflow.sklearn.log_model(model, 'model')

2025/06/22 13:11:37 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

2025/06/22 13:11:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instea

🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/176849608785059458/runs/12e53d3e838a4a1da6527f0f7018fc9d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/176849608785059458


2025/06/22 13:11:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/22 13:11:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/176849608785059458/runs/1b363da5f5dd4eb9bb3418430e8aa0dd
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/176849608785059458


2025/06/22 13:11:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/176849608785059458/runs/7db69b00d7ea4529807474179fb7646d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/176849608785059458


#### registry of the model

In [9]:
model_name = "XGBClassifier"
run_id = input("Enter run id")
model_uri = f"runs:/{run_id}/model" # after the run_id/here comes the artifact path
mlflow.register_model(model_uri=model_uri, name=model_name)

Registered model 'XGBClassifier' already exists. Creating a new version of this model...
2025/06/22 13:12:35 WARNING mlflow.tracking._model_registry.fluent: Run with id 7db69b00d7ea4529807474179fb7646d has no artifacts at artifact path 'model', registering model based on models:/m-3320543df5684c309833b10f1129ed27 instead
2025/06/22 13:12:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGBClassifier, version 3
Created version '3' of model 'XGBClassifier'.


<ModelVersion: aliases=[], creation_timestamp=1750578155279, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1750578155279, metrics=None, model_id=None, name='XGBClassifier', params=None, run_id='7db69b00d7ea4529807474179fb7646d', run_link='', source='models:/m-3320543df5684c309833b10f1129ed27', status='READY', status_message=None, tags={}, user_id='', version='3'>

#### load the model

In [ ]:
model_name = "XGBClassifier"
model_version = 3
#way 1
#model_uri = f"models:/{model_name}/{model_version}"
#way 2
model_uri = f"models:/{model_name}@challenger"

loaded_model = mlflow.xgboost.load_model(model_uri=model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

#### production model

In [12]:
dev_model_uri = f"models:/{model_name}@challenger"
prod_model = 'anomaly-detection-prod'
client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=dev_model_uri, dst_name=prod_model)

Successfully registered model 'anomaly-detection-prod'.
Copied version '3' of model 'XGBClassifier' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1750578700514, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1750578700514, metrics=None, model_id=None, name='anomaly-detection-prod', params=None, run_id='7db69b00d7ea4529807474179fb7646d', run_link='', source='models:/XGBClassifier/3', status='READY', status_message=None, tags={}, user_id='', version='1'>

#### download production model

In [14]:
import mlflow.xgboost


model_uri = f"models:/{prod_model}@champion"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])